# Time-series forecasting with `statsmodels`
presented by Craig Nilson (craig.t.nilson@gmail.com)

## Outline

- Data
 - Import and isolate time-series data
 - Test data for stationarity
- Model
 - Build a forecasting model
 - Review performance
- Wrap-up
- Further learning (if time allows)

## Initialize Python libraries

In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

## Import  dataset

Access toy dataset and view documentation.

In [ ]:
data = sm.datasets.macrodata
print(data.DESCRLONG, data.NOTE)

Load data into a `pandas` dataframe.

In [ ]:
df = data.load_pandas().data
df.tail(10)

Combine date columns, select a variable, and preview plot.

In [ ]:
# combine date columns into one
date = df['year'].map(int).map(str) + '-q' + df['quarter'].map(int).map(str)

# read a single variable into a pandas series, indexing with the newly created dates
series = pd.Series(df['infl'].values, index=pd.to_datetime(date)+pd.tseries.offsets.QuarterEnd())

# plot series
fig = series.plot(figsize=(16,4))

Which way do *you* think the inflation will swing next?

## Examine a time series
- https://www.analyticsvidhya.com/blog/2018/09/non-stationary-time-series-python/
- https://www.mathworks.com/help/econ/trend-stationary-vs-difference-stationary.html
- https://www.mathworks.com/help/econ/autocorrelation-and-partial-autocorrelation.html

**Stationarity**: A stochastic process whose statistical properties do not change over time (e.g. mean, variance, etc.) is said to be *strictly* stationary. Since stationarity is an assumption underlying many statistical procedures used in time-series analysis, non-stationary data are often transformed to become strictly stationary.

Types of stationarity:
- **Trend-Stationary**: The mean trend is deterministic. Once the trend is estimated and removed from the data, the residual series is a strictly stationary stochastic process. Time series with a deterministic trend always revert to the trend in the long run (the effects of shocks are eventually eliminated). Forecast intervals have constant width.
- **Difference-Stationary**: The mean trend is stochastic. Differencing the series one or more times yields a strictly stationary stochastic process. Time series with a stochastic trend never recover from shocks to the system (the effects of shocks are permanent). Forecast intervals grow over time.

Stationarity tests:
- **Augmented Dickey-Fuller** unit root (stationarity) test.
 - $H_0$: series has a unit root; non-stationary
 - $H_1$: series does not have a unit root; stationary
- **Kwiatkowski-Phillips-Schmidt-Shin** test for trend-stationarity.
 - $H_0$: series is trend stationary
 - $H_1$: series is non-trend stationary

In [ ]:
def stationarity(series, alpha=0.05):
    adf_p = sm.tsa.adfuller(series)[1]
    kpss_p = sm.tsa.kpss(series, lags='auto')[1]
    
    if adf_p < alpha:
        print(f'ADF\tREJECTED H0 (p={adf_p:.3f})')
    else:
        print(f'ADF\tFAILED TO REJECT H0 (p={adf_p:.3f})')
    if kpss_p < alpha:
        print(f'KPSS\tREJECTED H0 (p={kpss_p:.3f})')
    else:
        print(f'KPSS\tFAILED TO REJECT H0 (p={kpss_p:.3f})')
        
# run test
stationarity(series)

Stationarity test outcomes:
- **Case 1**: ADF *fails* to reject $H_0$, KPSS rejects $H_0$ $\rightarrow$ series is not stationary
- **Case 2**: ADF rejects $H_0$, KPSS *fails* to reject $H_0$ $\rightarrow$ series is stationary
- **Case 3**: ADF *fails* to reject $H_0$, KPSS *fails* to reject $H_0$ $\rightarrow$ trend stationary, remove the trend to make series strict stationary
- **Case 4**: ADF rejects $H_0$, KPSS rejects $H_0$ $\rightarrow$ difference stationary, use differencing to make series stationary

Correlation plots:
- **Autocorrelation (ACF)**: The linear dependence of a variable with itself at two points in time, ignoring the time segments in between. For stationary processes, autocorrelation between any two observations only depends on the time lag $h$ between them.
- **Partial Autocorrelation (PACF)**: The autocorrelation between $y_t$ and $y_{t–h}$ after removing any linear dependence on $y_1, y_2, \dots, y_{t–h+1}$.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12,8))
fig = sm.graphics.tsa.plot_acf(series, ax=axes[0])
fig = sm.graphics.tsa.plot_pacf(series, ax=axes[1])

## Build a model

ARMA (**A**uto**R**egressive **M**oving **A**verage) model
- https://www.statsmodels.org/dev/examples/notebooks/generated/tsa_arma_0.html
---

Parameters or "order" ($p,q$):
- **AR**: The number of autoregressive terms to include in the model. For example, if $p=2$, the model will correlate all $y_{t-2}$ and $y_{t-1}$ observations with all $y_t$.
- **MA**: The number of moving average terms to include in the model. For example, if $q=2$, the model will correlate the average of all $y_{t-2}$ and $y_{t-1}$ observations with all $y_t$.

Note: The ACF and PACF plots from before can give an idea of what "order" to use.

In [ ]:
# fit model and report
model = sm.tsa.ARMA(series, order=(2,0), freq='Q-DEC')
result = model.fit()
display(result.summary())

# plot model performance
fig, ax = plt.subplots(figsize=(12,5))
fig = result.plot_predict(end='2020', ax=ax)

The Akaike information criterion (AIC) tries to select the model that most adequately describes an unknown, high dimensional reality. The Bayesian information criterion (BIC) tries to find the true model among the set of candidates. In both cases, lower scores are desired.

Rather than trying every combination of "order" for our model, we can employ a machine learning technique known as **hyperparameter tuning** to determine the "order" that optimizes a score (AIC, in this case).

In [ ]:
order = sm.tsa.arma_order_select_ic(series, ic='aic')['aic_min_order']
order

Let's try the fit again with our tuned parameters...

In [ ]:
# fit model and report
model = sm.tsa.ARMA(series, order=(4,1), freq='Q-DEC')
result = model.fit()
display(result.summary())

# plot model performance
fig, ax = plt.subplots(figsize=(12,5))
fig = result.plot_predict(end='2020', ax=ax)

## Wrap-up
- How did our model compare with our initial predictions?
- What are we assuming?
- What are we missing? How can we improve our model?

## Thank you!

## Further exploration...

ARIMA (**A**uto**R**egressive **I**ntegrated **M**oving **A**verage) model...

SARIMAX (**S**easonal **A**uto**R**egressive **I**ntegrated **M**oving **A**verage with e**X**ogenous regressors model) model
- https://www.statsmodels.org/dev/examples/notebooks/generated/statespace_sarimax_stata.html
- https://medium.com/datadriveninvestor/time-series-prediction-using-sarimax-a6604f258c56

In [ ]:
# fit model and report
model = sm.tsa.SARIMAX(series, order=(0,0,0), seasonal_order=(2,0,2,1), freq='Q-DEC')
result = model.fit()
display(result.summary())

# plot model performance
fig, ax = plt.subplots(figsize=(12,5))
predict = result.get_prediction()
predict_ci = predict.conf_int()
series.plot(figsize=(16,4), ax=ax, c='C1')
predict.predicted_mean.plot(ax=ax, c='C4')
ax.fill_between(predict_ci.index, predict_ci.iloc[:,0], predict_ci.iloc[:,1], color='k', alpha=0.1)

Brute force optimization

In [ ]:
from scipy.optimize import brute

def sarimax_obj(composite, endog):
    order = composite[:3]
    seasonal_order = composite[3:]
    try:
        fit = sm.tsa.SARIMAX(endog, order=order, seasonal_order=seasonal_order, freq='Q-DEC').fit()
        score = fit.aic
    except:
        score = 1e4
    return score

grid = (slice(0, 1+0, 1),   # p
        slice(0, 1+0, 1),   # d
        slice(0, 1+0, 1),   # q
        slice(1, 4+1, 1),   # P
        slice(0, 2+1, 1),   # D
        slice(0, 2+1, 1),   # Q
        slice(1, 4+1, 1))   # s

order = brute(sarimax_obj, grid, args=[series], finish=None)
order

Exogenous variables...

Train, Validation, and Test sets for time series...